In [ ]:
import graph_fourier_transform
import graph_ruggedness_de
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import networkx as nx
import os
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import matplotlib.cm as cm

### Local Dirichlet energy of the PTE 2NH catalytic efficiency map
The below cell computes and visualizises the local Dirichlet energy over the PTE 2NH catalytic efficiency combinatorial landscape as a KNN graph. 

In [ ]:
def remove_negative_sign(x, pos):
    return str(x).replace("-", "")

def format_yticks(x, pos):
    return f"{abs(x):.1f}"

dtype_spec = {'Genotype': str}
df = pd.read_csv('../data_files/PTE_catact_2NH.csv', dtype=dtype_spec)
seq_ls = df['Genotype'].tolist()
values = df['F'].tolist()
scaler = MinMaxScaler()
values = [val[0] for val in (scaler.fit_transform(np.array(values).reshape(-1,1)))]

G_h = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        hamming_edges=True)
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_h)

G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        hamming_edges=False)
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_k)

pos=nx.fruchterman_reingold_layout(G_h)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(8, 3))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], figure=fig)

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# For the first graph
values = [node[1]['value'] for node in G_k.nodes(data=True)]
min_val, max_val = min(values), max(values)
viridis = plt.cm.get_cmap('viridis', 10)
norm = colors.Normalize(vmin=min_val, vmax=max_val)
node_colors = [viridis(norm(value)) for value in values]
sm = cm.ScalarMappable(cmap=viridis, norm=norm)
sm.set_array([])
nx.draw(G_k, pos=pos, ax=ax1, node_color=node_colors, with_labels=False, edgecolors='black', node_size=75, width=0.15, edge_color='grey')
fig.colorbar(sm, ax=ax1, orientation='vertical', shrink=0.8, label='Normalised Fitness')

# Define the nodes for which you want to highlight connected edges
nodes_to_highlight = ["100000", "110100"]

# Find edges connected to these nodes
edges_to_highlight = []
for node in nodes_to_highlight:
    if node in G_k:
        edges_to_highlight.extend(list(G_k.edges(node)))

# Highlight the connected edges in black
nx.draw_networkx_edges(G_k, pos, ax=ax1, edgelist=edges_to_highlight, edge_color='black', width=1)

# For the second graph
values = [node[1]['local_dirichlet'] for node in G_k.nodes(data=True)]
min_val, max_val = min(values), max(values)
plasma = plt.cm.get_cmap('plasma', 10)
norm = colors.Normalize(vmin=min_val, vmax=max_val)
node_colors = [plasma(norm(value)) for value in values]
sm = cm.ScalarMappable(cmap=plasma, norm=norm)
sm.set_array([])
nx.draw(G_k, pos=pos, ax=ax2, node_color=node_colors, with_labels=False, edgecolors='black', node_size=75, width=0.15, edge_color='grey')
fig.colorbar(sm, ax=ax2, orientation='vertical', shrink=0.8, label='Local Dirichlet Energy')

# Find edges connected to these nodes
edges_to_highlight = []
for node in nodes_to_highlight:
    if node in G_k:
        edges_to_highlight.extend(list(G_k.edges(node)))

# Highlight the connected edges in black
nx.draw_networkx_edges(G_k, pos, ax=ax2, edgelist=edges_to_highlight, edge_color='black', width=1, node_size=0)

plt.savefig('figures/Figure_3/PTE_2NH_knn.pdf')
plt.show()

### Local Dirichlet energy of the PTE 2NP-butyrate catalytic efficiency map
The below cell computes and visualizises the local Dirichlet energy over the PTE 2-nitrophenol butyrate catalytic efficiency combinatorial landscape as a KNN graph. 

In [ ]:
def remove_negative_sign(x, pos):
    return str(x).replace("-", "")

def format_yticks(x, pos):
    return f"{abs(x):.1f}"

dtype_spec = {'Genotype': str}
df = pd.read_csv('../data_files/PTE_catact_butyrate.csv', dtype=dtype_spec)
seq_ls = df['Genotype'].tolist()
values = df['F'].tolist()
scaler = MinMaxScaler()
values = [val[0] for val in (scaler.fit_transform(np.array(values).reshape(-1,1)))]

G_h = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        hamming_edges=True)
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_h)

G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        hamming_edges=False)
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_k)

#pos=nx.fruchterman_reingold_layout(G_h)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(8, 3))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], figure=fig)

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# For the first graph
values = [node[1]['value'] for node in G_k.nodes(data=True)]
min_val, max_val = min(values), max(values)
viridis = plt.cm.get_cmap('viridis', 10)
norm = colors.Normalize(vmin=min_val, vmax=max_val)
node_colors = [viridis(norm(value)) for value in values]
sm = cm.ScalarMappable(cmap=viridis, norm=norm)
sm.set_array([])
nx.draw(G_k, pos=pos, ax=ax1, node_color=node_colors, with_labels=False, edgecolors='black', node_size=75, width=0.15, edge_color='grey')
fig.colorbar(sm, ax=ax1, orientation='vertical', shrink=0.8, label='Normalised Fitness')

# Define the nodes for which you want to highlight connected edges
nodes_to_highlight = ["000001", "010101"]

# Find edges connected to these nodes
edges_to_highlight = []
for node in nodes_to_highlight:
    if node in G_k:
        edges_to_highlight.extend(list(G_k.edges(node)))

# Highlight the connected edges in black
nx.draw_networkx_edges(G_k, pos, ax=ax1, edgelist=edges_to_highlight, edge_color='black', width=1)

# For the second graph
values = [node[1]['local_dirichlet'] for node in G_k.nodes(data=True)]
min_val, max_val = min(values), max(values)
plasma = plt.cm.get_cmap('plasma', 10)
norm = colors.Normalize(vmin=min_val, vmax=max_val)
node_colors = [plasma(norm(value)) for value in values]
sm = cm.ScalarMappable(cmap=plasma, norm=norm)
sm.set_array([])
nx.draw(G_k, pos=pos, ax=ax2, node_color=node_colors, with_labels=False, edgecolors='black', node_size=75, width=0.15, edge_color='grey')
fig.colorbar(sm, ax=ax2, orientation='vertical', shrink=0.8, label='Local Dirichlet Energy')

# Find edges connected to these nodes
edges_to_highlight = []
for node in nodes_to_highlight:
    if node in G_k:
        edges_to_highlight.extend(list(G_k.edges(node)))

# Highlight the connected edges in black
nx.draw_networkx_edges(G_k, pos, ax=ax2, edgelist=edges_to_highlight, edge_color='black', width=1, node_size=0)

plt.savefig('figures/Figure_3/PTE_butyrate_knn.pdf')
plt.show()